In [146]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [147]:
file_to_read = "Resources/AviationData_Weather.csv"

data_df = pd.read_csv(file_to_read, encoding='ISO-8859-1', low_memory=False)



data_df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87277,20211223104439,Accident,GAA22WA054,2021-12-21,"Auxerre, OF",United States,475047N,0032948W,NaN,NaN,...,Unknown,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
87278,20211228104450,Accident,WPR22LA069,2021-12-22,"Auburn, CA",United States,NaN,NaN,NaN,NaN,...,Positioning,"Fox Two, LLC",0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
87279,20211227104443,Accident,CEN22FA082,2021-12-26,"Hardy, AR",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
87280,20211230104454,Accident,ERA22LA097,2021-12-29,"Marathon, FL",United States,NaN,NaN,NaN,NaN,...,NaN,ExecAir,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN


In [148]:
data_df.loc[(data_df['Aircraft.Category']=='Airplane') & (data_df['Country']== 'United States') & (data_df['Investigation.Type'] == 'Accident')]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
5,20170710X52551,Accident,NYC79AA106,1979-09-17,"BOSTON, MA",United States,42.445277,-70.758333,NaN,NaN,...,NaN,Air Canada,NaN,NaN,1.0,44.0,VMC,Climb,Probable Cause,19-09-2017
7,20020909X01562,Accident,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,NaN,NaN,NaN,BLACKBURN AG STRIP,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,Takeoff,Probable Cause,01-01-1982
8,20020909X01561,Accident,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,NaN,NaN,N58,HANOVER,...,Business,NaN,0.0,0.0,0.0,2.0,IMC,Landing,Probable Cause,01-01-1982
12,20020917X02148,Accident,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,0.0,1.0,0.0,IMC,Cruise,Probable Cause,02-01-1983
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,Personal,NaN,1.0,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87264,20211215104408,Accident,CEN22LA074,2021-12-13,"South Bend, IN",United States,413952N,0862119W,3IN7,CHAIN-O-LAKES,...,Personal,Joseph Michael Krol,0.0,0.0,0.0,1.0,VMC,NaN,NaN,NaN
87265,20211216104418,Accident,CEN22LA079,2021-12-13,"Ellinwood, KS",United States,382224N,0983555W,1K6,ELLINWOOD MUNI,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,NaN,NaN,NaN
87270,20211221104428,Accident,WPR22LA065,2021-12-16,"Ivanhoe, CA",United States,362327N,1191342W,NaN,NaN,...,Personal,FERRARO NICHOLAS A,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
87271,20211228104446,Accident,CEN22LA083,2021-12-16,"Houston, TX",United States,293626N,0095932W,EFD,NaN,...,Instructional,NaN,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN


In [149]:
# Keep rows where Investigation.Type = Accident, Country= United States, and Aircraft.Category= Airplane
before = len(data_df)
filtered_df = data_df.loc[(data_df['Aircraft.Category']=='Airplane') & (data_df['Country']== 'United States') & (data_df['Investigation.Type'] == 'Accident')]
print(f'{after} rows dropped. {len(filtered_df)} rows left.')

64638 rows dropped. 22644 rows left.


In [150]:

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22644 entries, 5 to 87277
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                22644 non-null  object 
 1   Investigation.Type      22644 non-null  object 
 2   Accident.Number         22644 non-null  object 
 3   Event.Date              22644 non-null  object 
 4   Location                22644 non-null  object 
 5   Country                 22644 non-null  object 
 6   Latitude                19397 non-null  object 
 7   Longitude               19387 non-null  object 
 8   Airport.Code            15799 non-null  object 
 9   Airport.Name            16316 non-null  object 
 10  Injury.Severity         22613 non-null  object 
 11  Aircraft.damage         22369 non-null  object 
 12  Aircraft.Category       22644 non-null  object 
 13  Registration.Number     22644 non-null  object 
 14  Make                    22638 non-null

In [151]:
filtered_df.shape

(22644, 31)

In [153]:
# Change dates to datetime 

filtered_df.loc[:,'Event.Date'] = pd.to_datetime(filtered_df['Event.Date'])
filtered_df.loc[:,'Publication.Date'] = pd.to_datetime(filtered_df['Publication.Date'])



C:\Users\wero\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [154]:
filtered_df.dtypes

Event.Id                          object
Investigation.Type                object
Accident.Number                   object
Event.Date                datetime64[ns]
Location                          object
Country                           object
Latitude                          object
Longitude                         object
Airport.Code                      object
Airport.Name                      object
Injury.Severity                   object
Aircraft.damage                   object
Aircraft.Category                 object
Registration.Number               object
Make                              object
Model                             object
Amateur.Built                     object
Number.of.Engines                float64
Engine.Type                       object
FAR.Description                   object
Schedule                          object
Purpose.of.flight                 object
Air.carrier                       object
Total.Fatal.Injuries             float64
Total.Serious.In

In [155]:
filtered_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,22068.000000,19731.000000,19751.000000,20095.000000,21930.000000
mean,1.117546,0.354518,0.279834,0.224932,3.326721
std,0.343325,1.955085,1.554648,0.798105,20.101464
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,228.000000,137.000000,50.000000,576.000000


In [156]:
# percent of missing data per column
pct_missing = filtered_df.isnull().mean()

pct_missing_df = pd.DataFrame(pct_missing, columns=['Missing'])
pct_missing_df.sort_values('Missing', ascending = False, inplace = True)

# columns with more than 10% missing values
print(pct_missing_df[pct_missing_df['Missing'] > 0.1])

                         Missing
Schedule                0.933713
Broad.phase.of.flight   0.725314
Air.carrier             0.606474
Airport.Code            0.302288
Airport.Name            0.279456
Longitude               0.143835
Latitude                0.143393
Total.Fatal.Injuries    0.128643
Total.Serious.Injuries  0.127760
Total.Minor.Injuries    0.112568


In [157]:
# Check for duplicate rows
print(filtered_df.duplicated().sum())

0


In [159]:
# value counts for all columns 
for col in filtered_df.columns:
    print(filtered_df[col].value_counts().nlargest(10))
    print('--------------------------\n')

20001212X22846    2
20020917X02154    2
20020917X03351    2
20020917X03553    2
20061015X01521    2
20020917X04070    2
20020917X04312    2
20001212X20487    2
20020917X03176    2
20020917X02402    2
Name: Event.Id, dtype: int64
--------------------------

Accident    22644
Name: Investigation.Type, dtype: int64
--------------------------

NYC79AA106    1
WPR15CA074    1
CEN15FA136    1
CEN15FA135    1
ERA15CA120    1
ERA15CA117    1
ANC15LA010    1
ANC15FA009    1
WPR15CA097    1
WPR15CA095    1
Name: Accident.Number, dtype: int64
--------------------------

1982-05-16    23
1982-07-09    21
1982-06-19    20
1982-04-18    19
1982-05-02    19
1982-03-13    19
1982-04-25    18
1982-05-01    18
1982-07-24    18
1982-10-03    18
Name: Event.Date, dtype: int64
--------------------------

Anchorage, AK      90
Talkeetna, AK      67
Palmer, AK         64
Phoenix, AZ        64
Fairbanks, AK      62
Atlanta, GA        53
Reno, NV           50
Houston, TX        50
Miami, FL          48
San Ant

In [103]:
# Drop columns with 50% missing values or more

cols_to_drop = list(pct_missing_df[pct_missing_df['Missing'] > 0.5].index)
filtered_df.drop(columns = cols_to_drop, axis =1, inplace = True)
print(cols_to_drop)

['Schedule', 'Air.carrier', 'FAR.Description', 'Aircraft.Category', 'Longitude', 'Latitude']
